In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
!pip install transformers torch
!pip install -q datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Required imports
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from tqdm import tqdm
import json
from collections import defaultdict
from huggingface_hub import login

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Load data
df = pd.read_csv('/content/gdrive/MyDrive/Projects/Emotion Analysis/emotion_analysis_results_combined_dominant_emotion_tokenizer2_affin.csv')
df.head()

,Search Term,Title,Author,Upvotes,Comments,Post URL,Created,post_content,Year,Hour,Topic,Probability,cleaned_post_content,Emotions
0,Auditory Processing Disorder,Language processing disorder (undiagnosed),glitterface3000,5,5,https://www.reddit.com/r/neurodiversity/commen...,2024-10-09 01:24:56,Any advice for language processing disorder?\n...,2024,1,7,0.695524,advice language processing disorder hello mom ...,{'Transformer': [{'outputs': [{'label': 'neutr...
1,Auditory Processing Disorder,Sensory Processing Disorder,Mrsethylmercury,24,13,https://www.reddit.com/r/neurodiversity/commen...,2023-05-14 00:39:35,I have always had sensory issues and been diag...,2023,0,5,0.994210,always sensory issue diagnose migraine even im...,{'Transformer': [{'outputs': [{'label': 'confu...
2,Auditory Processing Disorder,why Do some people have SPD(sensory processing...,Purple_Friend,16,5,https://www.reddit.com/r/neurodiversity/commen...,2022-02-08 22:12:21,"I have alot of sensory issues with sound, feel...",2022,22,0,0.818453,alot sensory issue sound feel hearing people s...,{'Transformer': [{'outputs': [{'label': 'neutr...
3,Auditory Processing Disorder,Anyone an analyst with a sensory and learning ...,HibiscusSabdariffa33,12,2,https://www.reddit.com/r/neurodiversity/commen...,2022-03-31 18:23:30,People in interviews don’t understand how I sa...,2022,18,0,0.985818,people interview dont understand say analytica...,{'Transformer': [{'outputs': [{'label': 'neutr...
4,Auditory Processing Disorder,People with Sensory Processing Disorder: How d...,iiHulkGirlii,11,13,https://www.reddit.com/r/neurodiversity/commen...,2019-12-30 02:47:40,My mom wants me to be social when I’m overwhel...,2019,2,0,1.000000,mom want social im overwhelmed wont let wear h...,{'Transformer': [{'outputs': [{'label': 'disap...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4841 entries, 0 to 4840
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Search Term           4841 non-null   object 
 1   Title                 4841 non-null   object 
 2   Author                4841 non-null   object 
 3   Upvotes               4841 non-null   int64  
 4   Comments              4841 non-null   int64  
 5   Post URL              4841 non-null   object 
 6   Created               4841 non-null   object 
 7   post_content          4841 non-null   object 
 8   Year                  4841 non-null   int64  
 9   Hour                  4841 non-null   int64  
 10  Topic                 4841 non-null   int64  
 11  Probability           4841 non-null   float64
 12  cleaned_post_content  4841 non-null   object 
 13  Emotions              4841 non-null   object 
dtypes: float64(1), int64(5), object(8)
memory usage: 529.6+ KB


In [ ]:
# Check for missing values
print("\nMissing values:")
print(df.isnull().sum())


Missing values:
Search Term             0
Title                   0
Author                  0
Upvotes                 0
Comments                0
Post URL                0
Created                 0
post_content            0
Year                    0
Hour                    0
Topic                   0
Probability             0
cleaned_post_content    0
Emotions                0
dtype: int64


In [ ]:
# Basic exploration
print("Dataset Shape:", df.shape)
print("\nColumns:", df.columns.tolist())

Dataset Shape: (4841, 14)

Columns: ['Search Term', 'Title', 'Author', 'Upvotes', 'Comments', 'Post URL', 'Created', 'post_content', 'Year', 'Hour', 'Topic', 'Probability', 'cleaned_post_content', 'Emotions']


In [ ]:
# Basic statistics
print("\nBasic Statistics:")
df.describe()


Basic Statistics:


,Upvotes,Comments,Year,Hour,Topic,Probability
count,4841.000000,4841.000000,4841.000000,4841.000000,4841.000000,4841.000000
mean,43.930386,17.395373,2021.925842,12.419955,2.177856,0.904348
std,243.187133,53.274704,2.233127,7.231634,2.353030,0.127631
min,0.000000,0.000000,2011.000000,0.000000,0.000000,0.309379
25%,3.000000,3.000000,2021.000000,6.000000,0.000000,0.839974
50%,7.000000,6.000000,2023.000000,14.000000,2.000000,0.966864
75%,18.000000,14.000000,2024.000000,19.000000,3.000000,1.000000
max,9593.000000,1328.000000,2024.000000,23.000000,10.000000,1.000000


In [ ]:
# Keep only required columns
columns_to_keep = [ 'Search Term', 'Title', 'Author', 'Upvotes', 'Comments', 'Post URL', 'Created', 'post_content',]
df = df[columns_to_keep]

In [ ]:
df.head()

,Search Term,Title,Author,Upvotes,Comments,Post URL,Created,post_content
0,Auditory Processing Disorder,Language processing disorder (undiagnosed),glitterface3000,5,5,https://www.reddit.com/r/neurodiversity/commen...,2024-10-09 01:24:56,Any advice for language processing disorder?\n...
1,Auditory Processing Disorder,Sensory Processing Disorder,Mrsethylmercury,24,13,https://www.reddit.com/r/neurodiversity/commen...,2023-05-14 00:39:35,I have always had sensory issues and been diag...
2,Auditory Processing Disorder,why Do some people have SPD(sensory processing...,Purple_Friend,16,5,https://www.reddit.com/r/neurodiversity/commen...,2022-02-08 22:12:21,"I have alot of sensory issues with sound, feel..."
3,Auditory Processing Disorder,Anyone an analyst with a sensory and learning ...,HibiscusSabdariffa33,12,2,https://www.reddit.com/r/neurodiversity/commen...,2022-03-31 18:23:30,People in interviews don’t understand how I sa...
4,Auditory Processing Disorder,People with Sensory Processing Disorder: How d...,iiHulkGirlii,11,13,https://www.reddit.com/r/neurodiversity/commen...,2019-12-30 02:47:40,My mom wants me to be social when I’m overwhel...


In [ ]:
# Remove duplicates
#df_cleaned = df.drop_duplicates(subset=['post_content'])
#print(f"Removed {len(df) - len(df_cleaned)} duplicate rows")
#df = df_cleaned

In [ ]:
# Text length analysis
#df['text_length'] = df['cleaned_post_content'].str.len()
#df['original_text_length'] = df['post_content'].str.len()

In [ ]:
# Visualize text length distributions
#plt.figure(figsize=(15, 5))
#plt.subplot(1, 2, 1)
#sns.histplot(data=df[df['text_length'] < df['text_length'].quantile(0.95)], x='text_length', bins=50)
#plt.title('Cleaned Text Length Distribution')

In [ ]:
def clean_text(text):
    if not isinstance(text, str):
        return ''

    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

In [ ]:
# Clean post_content
print("Cleaning post content...")
df['cleaned_post_content'] = df['post_content'].apply(clean_text)

# Remove stopwords (optional - depending on your needs)
print("Removing stopwords...")
df['cleaned_post_content_no_stop'] = df['cleaned_post_content'].apply(remove_stopwords)

# Display sample of cleaned text
print("\nSample of original vs cleaned text:")
for i in range(2):
    print(f"\nOriginal: {df['post_content'].iloc[i][:200]}")
    print(f"Cleaned: {df['cleaned_post_content'].iloc[i][:200]}")

Cleaning post content...


<ipython-input-14-a844b43de0b4>:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


Removing stopwords...

Sample of original vs cleaned text:

Original: Any advice for language processing disorder?

Hello, my mom recently told me I have a language processing disorder. (She’s a speech therapist) I have noticed all my life I struggled with not understan
Cleaned: any advice for language processing disorder hello my mom recently told me i have a language processing disorder she s a speech therapist i have noticed all my life i struggled with not understanding t

Original: I have always had sensory issues and been diagnosed with migraine. Even when I'm not having a migraine, I am extremely sensitive to light and easily become motion-sick. To the point where I can't go o
Cleaned: i have always had sensory issues and been diagnosed with migraine even when i m not having a migraine i am extremely sensitive to light and easily become motion sick to the point where i can t go outs


In [ ]:
# Add text length columns
df['original_length'] = df['post_content'].str.len()
df['cleaned_length'] = df['cleaned_post_content'].str.len()

# Display text length statistics
print("\nText length statistics:")
print(df[['original_length', 'cleaned_length']].describe())

# Remove extremely short texts (optional)
#min_length = 10  # adjust as needed
#df = df[df['cleaned_length'] > min_length]


Text length statistics:
       original_length  cleaned_length
count      4841.000000     4841.000000
mean       2450.134889     2304.098740
std        3215.203379     2885.415698
min          31.000000       30.000000
25%         715.000000      687.000000
50%        1384.000000     1335.000000
75%        2800.000000     2705.000000
max       32433.000000    31563.000000


In [ ]:
# Save the cleaned dataset
df.to_csv('preprocessed_dataset.csv', index=False)
print(f"\nPreprocessed dataset saved with {len(df)} rows")

# Display final dataset info
print("\nFinal dataset info:")
df.info()


Preprocessed dataset saved with 4841 rows

Final dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4841 entries, 0 to 4840
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Search Term                   4841 non-null   object
 1   Title                         4841 non-null   object
 2   Author                        4841 non-null   object
 3   Upvotes                       4841 non-null   int64 
 4   Comments                      4841 non-null   int64 
 5   Post URL                      4841 non-null   object
 6   Created                       4841 non-null   object
 7   post_content                  4841 non-null   object
 8   cleaned_post_content          4841 non-null   object
 9   cleaned_post_content_no_stop  4841 non-null   object
 10  original_length               4841 non-null   int64 
 11  cleaned_length                4841 non-null   int64 
dtypes: int64(4),

In [ ]:
df.head()

,Search Term,Title,Author,Upvotes,Comments,Post URL,Created,post_content,cleaned_post_content,cleaned_post_content_no_stop,original_length,cleaned_length
0,Auditory Processing Disorder,Language processing disorder (undiagnosed),glitterface3000,5,5,https://www.reddit.com/r/neurodiversity/commen...,2024-10-09 01:24:56,Any advice for language processing disorder?\n...,any advice for language processing disorder he...,advice language processing disorder hello mom ...,620,603
1,Auditory Processing Disorder,Sensory Processing Disorder,Mrsethylmercury,24,13,https://www.reddit.com/r/neurodiversity/commen...,2023-05-14 00:39:35,I have always had sensory issues and been diag...,i have always had sensory issues and been diag...,always sensory issues diagnosed migraine even ...,819,804
2,Auditory Processing Disorder,why Do some people have SPD(sensory processing...,Purple_Friend,16,5,https://www.reddit.com/r/neurodiversity/commen...,2022-02-08 22:12:21,"I have alot of sensory issues with sound, feel...",i have alot of sensory issues with sound feel ...,alot sensory issues sound feel hearing people ...,144,140
3,Auditory Processing Disorder,Anyone an analyst with a sensory and learning ...,HibiscusSabdariffa33,12,2,https://www.reddit.com/r/neurodiversity/commen...,2022-03-31 18:23:30,People in interviews don’t understand how I sa...,people in interviews don t understand how i sa...,people interviews understand say analytical ta...,675,662
4,Auditory Processing Disorder,People with Sensory Processing Disorder: How d...,iiHulkGirlii,11,13,https://www.reddit.com/r/neurodiversity/commen...,2019-12-30 02:47:40,My mom wants me to be social when I’m overwhel...,my mom wants me to be social when i m overwhel...,mom wants social overwhelmed let wear headphon...,347,342


In [ ]:
# Initialize model and tokenizer
model_name = "Clinical-AI-Apollo/Medical-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create NER pipeline with the specified model
ner_pipeline = pipeline(
    "token-classification",
    model=model_name,
    aggregation_strategy="simple",
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/736M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def test_pipeline():
    try:
        test_text = "45 year old woman diagnosed with CAD"
        result = ner_pipeline(test_text)
        print("Test results:")
        for entity in result:
            print(f"Entity: {entity['word']}, Type: {entity['entity_group']}, Score: {entity['score']:.3f}")
        return True
    except Exception as e:
        print(f"Pipeline test failed: {str(e)}")
        return False

# Run the test
pipeline_working = test_pipeline()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Test results:
Entity: 45 year old, Type: AGE, Score: 0.543
Entity: woman, Type: SEX, Score: 0.408
Entity: CAD, Type: DISEASE_DISORDER, Score: 0.346


In [ ]:
def process_texts_batch(df, batch_size=32):
    all_entities = []

    for i in tqdm(range(0, len(df), batch_size)):
        batch = df['cleaned_post_content'].iloc[i:i+batch_size]
        batch_results = []

        for text in batch:
            try:
                entities = ner_pipeline(text)
                batch_results.append(entities)
            except Exception as e:
                print(f"Error processing text: {str(e)}")
                batch_results.append([])

        all_entities.extend(batch_results)

    return all_entities

# Process all texts
print("Extracting medical entities from texts...")
df['extracted_entities'] = process_texts_batch(df)

Extracting medical entities from texts...


100%|██████████| 152/152 [7:53:44<00:00, 187.00s/it]


In [ ]:
def analyze_medical_entities(df):
    # Dictionary to store entity type statistics
    entity_stats = {}

    # Process each row's entities
    for entities in df['extracted_entities']:
        for entity in entities:
            entity_type = entity['entity_group']
            entity_text = entity['word']
            confidence = entity['score']

            if entity_type not in entity_stats:
                entity_stats[entity_type] = {
                    'count': 0,
                    'unique_entities': set(),
                    'high_confidence_entities': []  # entities with confidence > 0.7
                }

            entity_stats[entity_type]['count'] += 1
            entity_stats[entity_type]['unique_entities'].add(entity_text)

            if confidence > 0.7:
                entity_stats[entity_type]['high_confidence_entities'].append({
                    'text': entity_text,
                    'confidence': confidence
                })

    return entity_stats

# Analyze the extracted entities
entity_statistics = analyze_medical_entities(df)

In [ ]:
def display_entity_analysis(entity_stats):
    print("\nMedical Entity Analysis:")
    print("=" * 50)

    for entity_type, stats in entity_stats.items():
        print(f"\nEntity Type: {entity_type}")
        print("-" * 30)
        print(f"Total mentions: {stats['count']}")
        print(f"Unique entities: {len(stats['unique_entities'])}")
        print("\nTop 5 high confidence examples:")
        sorted_entities = sorted(stats['high_confidence_entities'],
                               key=lambda x: x['confidence'],
                               reverse=True)[:5]
        for entity in sorted_entities:
            print(f"- {entity['text']} (confidence: {entity['confidence']:.3f})")

# Display the analysis
display_entity_analysis(entity_statistics)


Medical Entity Analysis:

Entity Type: DISEASE_DISORDER
------------------------------
Total mentions: 15459
Unique entities: 2996

Top 5 high confidence examples:
- attention deficit hyperactivity disorder (confidence: 0.952)
- auditory processing disorder (confidence: 0.945)
- sensorineural hearing loss (confidence: 0.943)
- auditory processing disorder (confidence: 0.940)
- ocd ptsd (confidence: 0.938)

Entity Type: NONBIOLOGICAL_LOCATION
------------------------------
Total mentions: 2063
Unique entities: 902

Top 5 high confidence examples:
- specialist paediatrics area (confidence: 0.813)
- johns hopkins neuro psych dept (confidence: 0.792)
- doctors (confidence: 0.774)
- nearest children s hospital (confidence: 0.770)
- local adhd centre (confidence: 0.749)

Entity Type: SIGN_SYMPTOM
------------------------------
Total mentions: 16064
Unique entities: 5445

Top 5 high confidence examples:
- disfluencies (confidence: 0.947)
- pain (confidence: 0.945)
- short of breath (confiden

In [ ]:
def create_entity_columns(df):
    # Get unique entity types
    entity_types = set()
    for entities in df['extracted_entities']:
        for entity in entities:
            entity_types.add(entity['entity_group'])

    # Create columns for each entity type
    for entity_type in entity_types:
        df[f'entity_{entity_type}'] = df['extracted_entities'].apply(
            lambda x: [entity['word'] for entity in x
                      if entity['entity_group'] == entity_type
                      and entity['score'] > 0.5]
        )

    return df

# Create entity-specific columns
df = create_entity_columns(df)

In [ ]:
# Save to CSV
output_file = 'medical_entities_analysis.csv'
df.to_csv(output_file, index=False)
print(f"\nResults saved to {output_file}")

# Save detailed statistics to JSON
import json
import numpy as np

def convert_to_serializable(obj):
    if isinstance(obj, np.float32):
        return float(obj)
    if isinstance(obj, set):
        return list(obj)
    return obj

# Modified JSON saving code
with open('entity_statistics.json', 'w') as f:
    serializable_stats = {}
    for entity_type, stats in entity_statistics.items():
        serializable_stats[entity_type] = {
            'count': convert_to_serializable(stats['count']),
            'unique_entities': convert_to_serializable(stats['unique_entities']),
            'high_confidence_entities': [
                {
                    'text': item['text'],
                    'confidence': float(item['confidence'])  # Convert to regular float
                }
                for item in stats['high_confidence_entities']
            ]
        }
    json.dump(serializable_stats, f, indent=2)

print("Entity statistics saved successfully!")


Results saved to medical_entities_analysis.csv
Entity statistics saved successfully!
